In [ ]:
pip install jupyterlab-dash
jupyter labextension install jupyterlab-dash

In [10]:
import jupyter_server_proxy

In [11]:
# Imports
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go

In [12]:
# Load and preprocess data
df = pd.read_csv(
    'https://gist.githubusercontent.com/chriddyp/'
    'cb5392c35661370d95f300086accea51/raw/'
    '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
    'indicators.csv')

available_indicators = df['Indicator Name'].unique()

In [13]:
# Build AppViewer 
from jupyterlab_dash import AppViewer
viewer = AppViewer()

In [14]:
# from jupyter_dash import JupyterDash
# c = JupyterDash.infer_jupyter_proxy_config()
# print(c)

In [15]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    )
])

# Callbacks
@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('xaxis-column', 'value'),
     dash.dependencies.Input('yaxis-column', 'value'),
     dash.dependencies.Input('xaxis-type', 'value'),
     dash.dependencies.Input('yaxis-type', 'value'),
     dash.dependencies.Input('year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]
    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 1,
                'color': 'blue',
                'line': {'width': 2}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest',
        )
            }

viewer.show(app)

In [16]:
import socket
def find_open_port():
    s = socket.socket()
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind(("", 0))
    _, port = s.getsockname()
    s.close()

    return port

In [17]:
port = find_open_port()

In [18]:
socket.getfqdn()

'jupyter-tonycastronova'

In [19]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# # Run app and display result inline in the notebook
import os
#base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], 1234)
#url = 'https://jupyterhub.cuahsi.org/{}'.format(base_prefix)
#app.server_url = url
# app.config.requests_pathname_prefix = base_prefix
port = find_open_port()
app.run_server(mode='external', host=f'https://jupyterhub.cuahsi.org/user/tonycastronova/proxy/{port}', port=port)

ConnectionError: HTTPConnectionPool(host='https', port=80): Max retries exceeded with url: //jupyterhub.cuahsi.org/user/tonycastronova/proxy/37399:37399/_alive_287dc8b2-ff24-4cb9-9bf4-47d0c8442c67 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7faf44297d10>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
os.environ['JUPYTERHUB_SERVICE_PREFIX']

In [ ]:
app.server_url

Exception in thread Thread-55:
Traceback (most recent call last):
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/home/jovyan/data/conda-envs/covid19/lib/python3.7/site-packages/retrying.py", line 200, in 

In [20]:
from IPython import display
import os

def show_app(app, port = 9999, 
             width = 700, 
             height = 350, 
             offline = False,
            in_binder = True):

    host = 'https://jupyterhub.cuahsi.org'
    base_prefix = '{}proxy/{}/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)
    url = '{}{}'.format(host, base_prefix)
    print(url)
    
    iframe = '<a href="{url}" target="_new">Open in new window</a><hr><iframe src="{url}" width={width} height={height}></iframe>'.format(url = url, 
                                                                                  width = width, 
                                                                                  height = height)
    
    display.display_html(iframe, raw = True)
    
    return app.run_server(debug=False, # needs to be false in Jupyter
                          host = '0.0.0.0',
                          proxy=f'http://0.0.0.0::{url}',
                          port=80)

In [21]:
import dash_colorscales
import dash
import dash_html_components as html
import json

def get_app(port):
    app = dash.Dash('', requests_pathname_prefix=f'/user/tonycastronova/proxy/{port}/')

    app.scripts.config.serve_locally = False

    app.layout = html.Div([
        dash_colorscales.DashColorscales(
            id='colorscale-picker',
            nSwatches=7,
            fixSwatches=True
        ),
        html.P(id='output', children='')
    ])

    @app.callback(
            dash.dependencies.Output('output', 'children'),
            [dash.dependencies.Input('colorscale-picker', 'colorscale')])
    def display_output(colorscale):
        return json.dumps(colorscale)
    
    return app

In [22]:
import socket
def find_open_port():
    s = socket.socket()
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind(("", 0))
    _, port = s.getsockname()
    s.close()

    return port




In [23]:
port = find_open_port()
app = get_app(port)


In [24]:
show_app(app, port=port)

https://jupyterhub.cuahsi.org/user/tonycastronova/proxy/33145/


Open in new window

ProxyError: port: 80 is incompatible with the proxy:
    http://0.0.0.0::https://jupyterhub.cuahsi.org/user/tonycastronova/proxy/33145/
To see your app at https://jupyterhub.cuahsi.org/user/tonycastronova/proxy/33145/,
you must use port: None

In [25]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
#!pip install JupyterDash
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
!pip install jupyter-dash

In [ ]:
!jupyter labextension install jupyterlab-dash

OSError: Unable to communicate with the jupyter_dash notebook or JupyterLab 
extension required to infer Jupyter configuration.

In [1]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    base_url = 'https://jupyterhub.cuahsi.org'
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

In [2]:
import yaml

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature

output_notebook()

Loading BokehJS ...

In [3]:

def bkapp(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = ColumnDataSource.from_df(data)

    slider = Slider(start=0, end=30, value=0, step=1, title="Smoothing by N Days")
    slider.on_change('value', callback)

    doc.add_root(column(slider, plot))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [4]:
import os
import urllib
show(bkapp, notebook_url=remote_jupyter_proxy_url)  